<a href="https://colab.research.google.com/github/kelvinmadeleke/kelvinmadeleke/blob/main/RockPaperScissors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import tensorflow as tf
from collections import Counter

# Counter moves mapping
counter_moves = {"R": "P", "P": "S", "S": "R"}

# Encoding moves for ML models
move_to_num = {"R": 0, "P": 1, "S": 2}
num_to_move = {0: "R", 1: "P", 2: "S"}

# Opponent Quincy (Cyclic pattern R, R, P, P, S)
def quincy(prev_play, opponent_history=[]):
    sequence = ["R", "R", "P", "P", "S"]
    if not opponent_history:
        return sequence[0]
    last_move = opponent_history[-1]
    next_index = (sequence.index(last_move) + 1) % 5
    return sequence[next_index]

# Opponent Abbey (Random choices with a bias)
def abbey(prev_play, opponent_history=[]):
    if prev_play:
        opponent_history.append(prev_play)

    if len(opponent_history) < 2:
        return random.choice(["R", "P", "S"])  # Start with full randomness

    # Count how often opponent plays R, P, or S
    move_counts = Counter(opponent_history)

    # Identify the opponent's most common move
    most_common_move = move_counts.most_common(1)[0][0]

    # Abbey counters the most common move
    counter_moves = {"R": "P", "P": "S", "S": "R"}
    predicted_move = counter_moves[most_common_move]

    # **Weighted randomness to avoid being exploited**
    weights = {"R": 0.3, "P": 0.3, "S": 0.4}  # Default bias
    if predicted_move == "R":
        weights = {"R": 0.2, "P": 0.4, "S": 0.4}  # Favor P and S
    elif predicted_move == "P":
        weights = {"R": 0.4, "P": 0.2, "S": 0.4}  # Favor R and S
    elif predicted_move == "S":
        weights = {"R": 0.4, "P": 0.4, "S": 0.2}  # Favor R and P

    # Choose move based on dynamic weights
    choice = random.choices(["R", "P", "S"], weights=[weights["R"], weights["P"], weights["S"]])[0]

    return choice
# Opponent Kris (Counters the player's last move)
def kris(prev_play, opponent_history=[]):
    if prev_play is None:
        return random.choice(["R", "P", "S"])  # Return random move when starting the game

    if prev_play:
        opponent_history.append(prev_play)

    # If the opponent repeats the last move, exploit it
    if len(opponent_history) > 1 and opponent_history[-1] == opponent_history[-2]:
        return counter_moves[opponent_history[-1]]  # Counter the repeated move

    # Otherwise, just counter the last move
    return counter_moves[prev_play]
# Opponent Mrugesh (Hidden pattern player)
def mrugesh(prev_play, opponent_history=[]):
    if len(opponent_history) < 4:
        return random.choice(["R", "P", "S"])
    pattern = "".join(opponent_history[-4:])
    if pattern in ["RRPP", "PPSS", "SSRR"]:
        return "R"
    return counter_moves[opponent_history[-1]]

# Create LSTM model
def create_lstm_model():
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(32, input_shape=(4, 1)),  # Using 4 previous moves
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

lstm_model = create_lstm_model()

def train_lstm_model(model, history):
    if len(history) < 5:
        return
    X, y = [], []
    for i in range(len(history) - 4):
        X.append([move_to_num[m] for m in history[i:i+4]])
        y.append(move_to_num[history[i+4]])
    X = np.array(X).reshape(-1, 4, 1)
    y = tf.keras.utils.to_categorical(y, num_classes=3)
    model.fit(X, y, epochs=10, verbose=0)

def predict_lstm(model, history):
    if len(history) < 4:
        return random.choice(["R", "P", "S"])
    input_data = np.array([move_to_num[m] for m in history[-4:]]).reshape(1, 4, 1)
    prediction = model.predict(input_data, verbose=0)
    predicted_move = np.argmax(prediction)
    return counter_moves[num_to_move[predicted_move]]

# Hybrid AI Player
def player(prev_play, opponent_history=[]):
    if prev_play:
        opponent_history.append(prev_play)

    if len(opponent_history) == 0:
        return random.choice(["R", "P", "S"])




    # Introduce a little randomness to avoid being too predictable
    if random.random() < 0.2:
        return random.choice(["R", "P", "S"])


    # Detect Quincy (Cyclic pattern R, R, P, P, S)
    if len(opponent_history) >= 5 and opponent_history[-5:] == ["R", "R", "P", "P", "S"]:
        return counter_moves[opponent_history[len(opponent_history) % 5]]

    # Detect Kris (Always counters player's last move)
    if len(opponent_history) > 1:
        if opponent_history[-1] == counter_moves[opponent_history[-2]]:
            return counter_moves[counter_moves[opponent_history[-1]]]

    # Detect Abbey (Random) → Use frequency-based prediction
    move_counts = Counter(opponent_history)
    if len(opponent_history) > 10:
        most_common_move = move_counts.most_common(1)[0][0]
        return counter_moves[most_common_move]

    # Detect Mrugesh (Hidden Pattern) → Use LSTM
    if len(opponent_history) >= 5:
        train_lstm_model(lstm_model, opponent_history)
        return predict_lstm(lstm_model, opponent_history)

    # Default to random if no pattern is detected
    return random.choice(["R", "P", "S"])

# Play Function
def play(player1, player2, num_games=1000):
    results = {"player1_wins": 0, "player2_wins": 0, "ties": 0}
    opponent_history = []

    for _ in range(num_games):
        move1 = player1("", opponent_history) if player1 == player else player1(None, opponent_history)
        move2 = player2("", opponent_history) if player2 == player else player2(None, opponent_history)

        if move1 == move2:
            results["ties"] += 1
        elif (move1 == "R" and move2 == "S") or (move1 == "P" and move2 == "R") or (move1 == "S" and move2 == "P"):
            results["player1_wins"] += 1
        else:
            results["player2_wins"] += 1

        opponent_history.append(move2)

    return results

# Run the simulation
num_games = 1000
opponents = {"Quincy": quincy, "Abbey": abbey, "Kris": kris, "Mrugesh": mrugesh}

for name, opponent in opponents.items():
    results = play(player, opponent, num_games)
    accuracy = (results["player1_wins"] / num_games) * 100
    print(f"Accuracy against {name}: {accuracy:.2f}%")


Accuracy against Quincy: 85.60%
Accuracy against Abbey: 37.00%
Accuracy against Kris: 33.10%
Accuracy against Mrugesh: 86.80%
